## Load data

In [47]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")
random_state = 12041500

In [48]:
df_train = pd.read_json("./data/synthetic_data_CTGANSynthesizer.json")
df_test = pd.read_json("./data/testset.json")

df_train.drop(columns=['fnlwgt'], inplace=True)
df_test.drop(columns=['fnlwgt'], inplace=True)

ratio_features = ["age", "capital-gain", "capital-loss", "hours-per-week"]
ordinal_features = ["education-num"]
nominal_features = ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'age', 'hours-per-week', 'capital-gain', 'capital-loss']
target = 'income'

In [49]:
def clean_data(X: pd.DataFrame) -> pd.DataFrame:
    X = X.reset_index(drop=True)
    cols = list(X.columns)
    X[cols] = X[cols].replace([" ?"], np.nan)
    X = X.dropna()
    def strip_str(x):
        if isinstance(x, str):
            return x.strip()
        else:
            return x
    X = X.applymap(strip_str)
    # X["relationship"] = X["relationship"].replace(["Husband", "Wife"], "Married")
    X["hours-per-week"] = pd.cut(
        x=X["hours-per-week"],
        bins=[0.9, 25, 39, 40, 55, 100],
        labels=["PartTime", "MidTime", "FullTime", "OverTime", "BrainDrain"],
    )
    X.age = pd.qcut(X.age, q=5)
    X["capital-gain"] = pd.cut(
        x=X["capital-gain"],
        bins=[-1, 0, 5000, 10000, 50000, 100000],
        labels=["NoGain", "LowGain", "MediumGain", "HighGain", "VeryHighGain"],
    )
    X["capital-loss"] = pd.cut(
        x=X["capital-loss"],
        bins=[-1, 0, 1000, 2000, 5000, 100000],
        labels=["NoLoss", "LowLoss", "MediumLoss", "HighLoss", "VeryHighLoss"],
    )

    return X

df_train = clean_data(df_train.dropna())
df_test = clean_data(df_test.dropna())

In [50]:
for col in df_train.columns:
    if df_train[col].dtype == "category":
        df_train[col] = df_train[col].astype("object")
        df_test[col] = df_test[col].astype("object")

## Train baseline

In [7]:
from sklearn.linear_model import LogisticRegression
from utils import create_model, train_and_evaluate, describe_model

In [8]:
clf = LogisticRegression(max_iter=1000, random_state=random_state)

In [9]:
## Train baseline
model = create_model(clf, nominal_features)
y_test, y_pred = train_and_evaluate(model, df_train, df_test, target, drop_na=True)
metrics = describe_model(y_test, y_pred, verbose=True)

Metric         Value               
Accuracy       0.8366259015365318
Precision      0.815639353554005
Recall         0.7010494485756561
F1             0.7318629810496294


## Fairness Evaluation

In [10]:
from utils import split_data
from utils_fairness import search_bias, calc_fairness_score, explain_detected_bias

In [11]:
X_train, y_train = split_data(df_train, target, drop_na=True)

In [12]:
model = create_model(clf, nominal_features)
model.fit(X_train, y_train)
probs = pd.Series(model.predict_proba(X_train)[:, 1]) # we select for target label

In [15]:
privileged_subset, _ = search_bias(X_train, y_train, probs, 1, penalty=1)

In [16]:
print(privileged_subset)

({'education-num': [11, 13, 14], 'capital-loss': ['NoLoss'], 'sex': ['Male'], 'race': ['Black'], 'occupation': ['Prof-specialty', 'Protective-serv']}, 7.357)


In [17]:
calc_fairness_score(df_train, privileged_subset[0].keys(), target, verbose=True)

Sensitive Attributes: ['capital-loss', 'education-num', 'occupation', 'race', 'sex']

                               Group Distance  Proportion  Counts   P-Value
        Exec-managerial, White, Male    0.486    0.042460    1585  0.00e+00
         Prof-specialty, White, Male    0.373    0.061052    2279  0.00e+00
                     Exec-managerial    0.409    0.067240    2510  0.00e+00
                Prof-specialty, Male    0.348    0.078464    2929  0.00e+00
               Exec-managerial, Male    0.482    0.051140    1909  0.00e+00
       NoLoss, Exec-managerial, Male    0.472    0.039326    1468  0.00e+00
              Exec-managerial, White    0.417    0.054971    2052  0.00e+00
                              Female   -0.118    0.371481   13867 2.38e-317
             NoLoss, Exec-managerial    0.396    0.051381    1918 2.51e-312
NoLoss, Exec-managerial, White, Male    0.479    0.032495    1213 3.98e-283

Weighted Mean Statistical Distance: 0.13175298419252776


In [18]:
explain_detected_bias(df_train, probs, target, privileged_subset[0])

Our detected privileged group has a size of 110, we observe 0.0455 as the average probability of earning >50k, but our model predicts 0.2014


## Fairness Metrics

In [19]:
from utils_fairness import transform_to_bias_dataset, describe_fairness, scan_and_calculate_fairness, plot_fairness_metrics

In [20]:
df_train_bias = transform_to_bias_dataset(df_train, list(privileged_subset[0].keys()), list(privileged_subset[0].values()), verbose=True)
df_test_bias = transform_to_bias_dataset(df_test, list(privileged_subset[0].keys()), list(privileged_subset[0].values()), verbose=True)

In [21]:
model = create_model(clf, nominal_features)
y_test, y_pred = train_and_evaluate(model, df_train, df_train, target, drop_na=True)
metrics = describe_fairness(df_train_bias[target], y_pred, list(privileged_subset[0].keys()), verbose=True)

Metric                          Value               
statistical_parity_difference   0.027316204577818276
average_odds_difference         -0.22859767869483066
equal_opportunity_difference    -0.41778127458693937
disparate_impact                1.2003188335706674
theil_index                     0.10603210355698797


In [ ]:
df_fairness_metrics, priviliged_subsets = pd.DataFrame(
    columns=[
        "statistical_parity_difference",
        "average_abs_odds_difference",
        "equal_opportunity_difference",
        "disparate_impact",
        "theil_index",
    ]
), {}

for i in [1e-17, 1e-10, 0.001, 0.01, 0.1, 0, 1, 5, 10, 25, 50, 100]:
    metrics, priv = scan_and_calculate_fairness(model, df_train, target, i)    
    df_fairness_metrics.loc[f"{i}"] = metrics.values()
    priviliged_subsets[f"{i}"] = priv

In [ ]:
df_fairness_metrics

In [ ]:
df_fairness_metrics.to_json("./results/fairness_metrics_synthetic.json")

## Mitigation

In [39]:
from tqdm import tqdm
from sklearn import clone
from aif360.algorithms.preprocessing import Reweighing
from utils_fairness import create_aif360_standardDataset, compute_metrics, reweight_mitigation
from utils import plot_metrics

In [52]:
def custom_one_hot_encoding(X_train, X_test, nominal_features):
    X_train = pd.get_dummies(X_train, columns=nominal_features)
    X_test = pd.get_dummies(X_test, columns=nominal_features)

    X_train, X_test = X_train.align(X_test, join='outer', axis=1, fill_value=0)

    X_train.index = X_train.index
    X_test.index = X_test.index
    # X_train[X_train.columns] = X_train[X_train.columns].astype(int)
    # Xtest[X_test.columns] = X_test[X_test.columns].astype(int)
    return X_train, X_test

_df_train, _df_test = custom_one_hot_encoding(df_train, df_test, [feature for feature in nominal_features if feature not in list(privileged_subset[0].keys())])

### Reweighting

In [54]:
# create (un)privileged groups
privileged_groups = [{key: 1 for key in list(privileged_subset[0].keys())}]
unprivileged_groups = [{key: 0 for key in list(privileged_subset[0].keys())}]

In [56]:
# convert dataset
train_dataset = create_aif360_standardDataset(_df_train.dropna(), [], target, 1, list(privileged_subset[0].keys()), list(privileged_subset[0].values()))
test_dataset = create_aif360_standardDataset(_df_test.dropna(), [], target, 1, list(privileged_subset[0].keys()), list(privileged_subset[0].values()))

In [57]:
RW = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
RW.fit(train_dataset)

train_dataset_reweight = RW.transform(train_dataset)

#### Baseline

In [58]:
model = clone(clf)
model.fit(train_dataset.features, train_dataset.labels.ravel())
y_pred = model.predict(test_dataset.features)

_ = describe_model(test_dataset.labels.ravel(), y_pred, verbose=True)

Metric         Value               
Accuracy       0.8015051740357478
Precision      0.7979767681615386
Recall         0.6082894522935671
F1             0.6231133374470346


In [23]:
y_pred = model.predict(train_dataset.features)
_ = describe_fairness(df_train_bias[target], y_pred, list(privileged_subset[0].keys()), verbose=True)

Metric                          Value               
statistical_parity_difference   0.27325293449977367
average_odds_difference         0.3160490306583643
equal_opportunity_difference    0.5220668823049727
disparate_impact                20.055737563922488
theil_index                     0.10899556247160261


#### Mitigated Model

In [59]:
# mitigated model
model = clone(clf)
model.fit(train_dataset_reweight.features, train_dataset_reweight.labels.ravel(), sample_weight=train_dataset_reweight.instance_weights)
y_pred = model.predict(test_dataset.features)

_ = describe_model(test_dataset.labels.ravel(), y_pred, verbose=True)

Metric         Value               
Accuracy       0.8026549597574998
Precision      0.8029206959222099
Recall         0.6096403351718331
F1             0.6249891623141524


In [60]:
y_pred = model.predict(train_dataset.features)
_ = describe_fairness(df_train_bias[target], y_pred, list(privileged_subset[0].keys()), verbose=True)

Metric                          Value               
statistical_parity_difference   -0.00918177177345883
average_odds_difference         -0.16461033371776407
equal_opportunity_difference    -0.250432730133753
disparate_impact                0.9468423739431331
theil_index                     0.11570285629695416


#### Bias detection & mitigation until bias free

In [27]:
df_fairness_metrics = pd.DataFrame(
    columns=[
        "statistical_parity_difference",
        "average_abs_odds_difference",
        "equal_opportunity_difference",
        "disparate_impact",
        "theil_index",
    ]
)

In [ ]:
df_metrics = pd.DataFrame(
    columns=['acc', 'prec', 'rec', 'f1']
)

In [ ]:
weights = None

weights_hist = [weights]
max_iter = 10
for i in tqdm(range(max_iter)):
    X_train, y_train = split_data(df_train, target, True)
    X_test, y_test = split_data(df_test, target, True)
    weights, model_metrics, fair_metrics = reweight_mitigation(clf, nominal_features, target, X_train, y_train, X_test, y_test, penalty=1, sample_weights=weights)
    if model_metrics is None and fair_metrics is None and weights is None:
        break

    df_metrics.loc[f"mitigation_{i}"] = model_metrics.values()
    df_fairness_metrics.loc[f"mitigation_{i}"] = fair_metrics.values()
    weights_hist.append(weights)

### Fair Learning

In [61]:
from utils import prepare_data_fair_learning, plot_fairlearning_results
from utils_fairness import get_fair_learning_scoring
from sklearn.model_selection import GridSearchCV
from aif360.sklearn.preprocessing import LearnedFairRepresentations

In [66]:
df_train_bias = transform_to_bias_dataset(df_train, list(privileged_subset[0].keys()), list(privileged_subset[0].values()), verbose=True)
df_test_bias = transform_to_bias_dataset(df_train, list(privileged_subset[0].keys()), list(privileged_subset[0].values()), verbose=True)

X_train, y_train, X_test, y_test = prepare_data_fair_learning(df_train_bias, df_test_bias, nominal_features, target)

In [67]:
max_delta = get_fair_learning_scoring(list(privileged_subset[0].keys()))

In [68]:
lfr = LearnedFairRepresentations(
    list(privileged_subset[0].keys()),
    n_prototypes=25,
    max_iter=100,
    random_state=random_state,
)

In [69]:
params = {
    "reconstruct_weight": [1e-2, 1e-3, 1e-4],
    "target_weight": [100, 1000],
    "fairness_weight": [0, 100, 1000],
}

In [70]:
grid = GridSearchCV(lfr, params, scoring=max_delta, cv=3, n_jobs=-1).fit(
    X_train, y_train, priv_group=(1,) * len(list(privileged_subset[0].keys()))
)
res = pd.DataFrame(grid.cv_results_)

/Users/maxkleinegger/Documents/TUWien/2023WS/BA/.ba-thesis/lib/python3.10/site-packages/torch/_functorch/deprecated.py:61: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.vmap is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.vmap instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('vmap', 'torch.vmap')
/Users/maxkleinegger/Documents/TUWien/2023WS/BA/.ba-thesis/lib/python3.10/site-packages/torch/_functorch/deprecated.py:61: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.vmap is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.vmap instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/m

#### Baseline

In [72]:
model = clone(clf)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

_ = describe_model(y_test, y_pred, verbose=True)

Metric         Value               
Accuracy       0.8705296150446034
Precision      0.813527939120468
Recall         0.7641013252007482
F1             0.7845435554975462


In [75]:
y_pred = model.predict(X_train)
_ = describe_fairness(df_train_bias[target], y_pred, list(privileged_subset[0].keys()), verbose=True)

Metric                          Value               
statistical_parity_difference   0.02755801655557158
average_odds_difference         -0.22786158429608466
equal_opportunity_difference    -0.41620771046420146
disparate_impact                1.2020921214075249
theil_index                     0.1056484782584031


#### Using Grid itself

In [76]:
_ = describe_model(y_test, grid.predict(X_test), verbose=True)

Metric         Value               
Accuracy       0.860671327921991
Precision      0.8138180324606257
Recall         0.7181291095838941
F1             0.7498191506199834


In [77]:
_ = describe_fairness(y_train, grid.predict(X_train), list(privileged_subset[0].keys()) ,verbose=True)

Metric                          Value               
statistical_parity_difference   0.03001424003868991
average_odds_difference         -0.07932585150440108
equal_opportunity_difference    -0.123078940466824
disparate_impact                1.300142400386899
theil_index                     0.12597404950862984


#### Transforming data

In [78]:
model = clone(clf)
model.fit(grid.transform(X_train), y_train)
y_pred = model.predict(X_test)

_ = describe_model(y_test, y_pred, verbose=True)

Metric         Value               
Accuracy       0.2044255136756945
Precision      0.10221275683784725
Recall                     0.5
F1             0.16972864768683274


In [79]:
_ = describe_fairness(y_train, model.predict(X_train), list(privileged_subset[0].keys()), verbose=True)

Metric                          Value               
statistical_parity_difference               0.0
average_odds_difference                     0.0
equal_opportunity_difference                0.0
disparate_impact                            1.0
theil_index                     0.028907602900327994


##### Also Transforming test data

In [80]:
_ = describe_model(y_test, model.predict(grid.transform(X_test)), verbose=True)

Metric         Value               
Accuracy       0.8625197567574808
Precision      0.805220529474548
Recall         0.7390573449217117
F1             0.7641904608117651


In [81]:
_ = describe_fairness(y_train, model.predict(grid.transform(X_train)), list(privileged_subset[0].keys()) ,verbose=True)

Metric                          Value               
statistical_parity_difference   0.013506053848352156
average_odds_difference         -0.06612768025789179
equal_opportunity_difference    -0.06984002098085496
disparate_impact                1.0990443948879158
theil_index                     0.11691149079993556


Multiple Runs cannot be done, because of the dataset already needs to be one hot encoded for the mitigation, making the `search_bias` function unnecessary and returning not viable resolutions.

### Fair Learning

In [82]:
%matplotlib inline
# Load all necessary packages

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult
from aif360.algorithms.preprocessing.lfr import LFR

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
import numpy as np
from utils_fairness import create_aif360_standardDataset

In [83]:
# create (un)privileged groups
privileged_groups = [{key: 1 for key in list(privileged_subset[0].keys())}]
unprivileged_groups = [{key: 0 for key in list(privileged_subset[0].keys())}]

In [84]:
grid.best_params_

{'fairness_weight': 1000, 'reconstruct_weight': 0.001, 'target_weight': 100}

In [85]:
TR = LFR(unprivileged_groups=unprivileged_groups,
         privileged_groups=privileged_groups,
         k=5, Ax=0.001, Ay=100, Az=1000,
         verbose=1,
         seed=random_state
)

TR = TR.fit(train_dataset, maxiter=5000, maxfun=1000)

step: 0, loss: 78.20657769578119, L_x: 0.5457422165300079,  L_y: 0.6856380468772482,  L_z: 0.009642227265839843
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          240     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  7.82066D+01    |proj g|=  1.26856D+01
step: 250, loss: 69.53134829158694, L_x: 0.5456077422192046,  L_y: 0.6321676361535841,  L_z: 0.00631403906848631

At iterate    1    f=  6.95313D+01    |proj g|=  1.55235D+01
step: 500, loss: 62.72247517405184, L_x: 0.5436820946880235,  L_y: 0.548459567054186,  L_z: 0.007875974786538554
step: 750, loss: 57.278535815649136, L_x: 0.5447698629527601,  L_y: 0.5203337871745667,  L_z: 0.005244612328329512

At iterate    2    f=  5.72785D+01    |proj g|=  1.85726D+01
step: 1000, loss: 54.378108638912316, L_x: 0.5481890599350094,  L_y: 0.5199503747202057,  L_z: 0.0023825229778318135

At iterate    3    f=  5.43781D+01    |proj g|=  9.67687D+00

           * *

In [86]:
# Transform training data and align features
train_dataset_lfr = TR.transform(train_dataset)

In [88]:
cnt = {}
for i in train_dataset_lfr.labels.ravel():
    cnt[i] = cnt.get(i, 0) + 1

cnt

{0.0: 37329}

#### Baseline

In [89]:
model = clone(clf)
model.fit(train_dataset.features, train_dataset.labels.ravel())
y_pred = model.predict(test_dataset.features)

_ = describe_model(test_dataset.labels.ravel(), y_pred, verbose=True)

Metric         Value               
Accuracy       0.8015051740357478
Precision      0.7979767681615386
Recall         0.6082894522935671
F1             0.6231133374470346


In [101]:
y_pred = model.predict(train_dataset.features)
_ = describe_fairness(df_train_bias[target], y_pred, list(privileged_subset[0].keys()), verbose=True)

Metric                          Value               
statistical_parity_difference   -0.034483609300235224
average_odds_difference         -0.27993330920799864
equal_opportunity_difference    -0.46682402307894044
disparate_impact                0.8193715703321012
theil_index                     0.11820128511811895


#### Mitigation

In [102]:
_ = describe_model(df_test_bias[target], [0]*len(df_test_bias[target]), verbose=True)

Metric         Value               
Accuracy       0.7955744863243055
Precision      0.39778724316215275
Recall                     0.5
F1             0.44307517865934626


In [103]:
_ = describe_fairness(df_train_bias[target], np.array([0]*len(df_train_bias)), list(privileged_subset[0].keys()), verbose=True)

Metric                          Value               
statistical_parity_difference               0.0
average_odds_difference                     0.0
equal_opportunity_difference                0.0
disparate_impact                            0.0
theil_index                     0.22869080098773423


### Disparate Impact Remover

In [104]:
import numpy as np
from tqdm import tqdm

from aif360.algorithms.preprocessing import DisparateImpactRemover

In [108]:
train_dataset.feature_names.index('sex', 'race')

TypeError: slice indices must be integers or have an __index__ method

In [112]:
index = train_dataset.feature_names.index(list(privileged_subset[0].keys())[0])

In [113]:
fairness_metrics = []
utility_metrics = []
for level in tqdm(np.linspace(0, 1, 10)):
    di = DisparateImpactRemover(repair_level=level)
    train_repd = di.fit_transform(train_dataset)
    test_repd = di.fit_transform(test_dataset)
    
    #X_tr = train_repd.features # 
    X_tr = np.delete(train_repd.features, index, axis=1)
    #X_te = test_repd.features # 
    X_te = np.delete(test_repd.features, index, axis=1)
    y_tr = train_repd.labels.ravel()
    
    model = clone(clf)
    model.fit(X_tr, y_tr)

    utility_metrics.append(describe_model(test_repd.labels.ravel(), model.predict(X_te), verbose=False))
    fairness_metrics.append(describe_fairness(df_train_bias[target], model.predict(X_tr), list(privileged_subset[0].keys()), verbose=False))

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:49<00:00,  4.96s/it]


In [114]:
fairness_metrics

[{'statistical_parity_difference': 0.02034005114689713,
  'average_odds_difference': -0.07412497327729684,
  'equal_opportunity_difference': -0.10039339103068445,
  'disparate_impact': 1.1598146875827633,
  'theil_index': 0.12529740460945935},
 {'statistical_parity_difference': 0.02034005114689713,
  'average_odds_difference': -0.07412497327729684,
  'equal_opportunity_difference': -0.10039339103068445,
  'disparate_impact': 1.1598146875827633,
  'theil_index': 0.12529740460945935},
 {'statistical_parity_difference': 0.02034005114689713,
  'average_odds_difference': -0.07412497327729684,
  'equal_opportunity_difference': -0.10039339103068445,
  'disparate_impact': 1.1598146875827633,
  'theil_index': 0.12529740460945935},
 {'statistical_parity_difference': 0.02034005114689713,
  'average_odds_difference': -0.07412497327729684,
  'equal_opportunity_difference': -0.10039339103068445,
  'disparate_impact': 1.1598146875827633,
  'theil_index': 0.12529740460945935},
 {'statistical_parity_di